# Testando paralelização em GPUS através da biblioteca Numba

## Multiplicando vetores (CPU)

### Declarando a função

In [2]:
import numpy as np

def mulVectorsCPU(a:list[np.float64], b:list[np.float64]):
    if a.size != b.size: raise RuntimeError('mulVectorsCPU() expects two vectors (lists) of equal length!')

    # Initializing vector to be returned with zeroes (with default dtype -> np.float64)
    c = np.zeros(a.size)
    for i in range(b.size): c[i] = a[i] * b[i]
    return c

### Inicializando vetores para testes

Inicializando dois vetores **`a`** e **`b`** com **100.000.000 de floats randomizados** (no **intervalo [-10, +10]**).

> Esta parte de geração costuma demorar 3m40s para executar num Ryzen 7 7700X @ 5.4 GHz!

In [3]:
N = 100000000
MIN_VAL = -10.0
MAX_VAL = 10.0

# Initializing vectors with zeroes (with default dtype -> np.float64)
a = np.zeros(N)
b = np.zeros(N)

# Filling vectors with random floats
for i in range(N):
    a[i] = np.random.uniform(MIN_VAL, MAX_VAL)
    b[i] = np.random.uniform(MIN_VAL, MAX_VAL)

### Rodando e cronometrando a função

Utilizando o código abaixo podemos avaliar o **tempo de execução** da função **`mulVectorsCPU()`**, que roda na CPU.

Em **dez execuções** de teste (com o mesmo input), temos:

- Tempo médio: ~14,6612 segundos
- Tempo mínimo: ~11,6958 segundos
- Tempo máximo: ~22,6586 segundos

In [24]:
from timeit import default_timer as timer

print('Multiplying vectors...')

startT = timer() # Startng our timer
ab = mulVectorsCPU(a, b)
endT = timer() # Endng our timer
execTime = endT - startT

print(f'Vectors multiplied with success!\nExecution time of {execTime} seconds\n\nResults:')

print(f'a[:5] = {a[:5]}; a[-5:] = {a[-5:]}')
print(f'b[:5] = {b[:5]}; b[-5:] = {b[-5:]}')
print(f'a*b[:5] = {ab[:5]}; a*b[-5:] = {ab[-5:]}')

Multiplying vectors...
Vectors multiplied with success!
Execution time of 18.97332693899807 seconds

Results:
a[:5] = [-5.57351108 -2.16060523  2.43268902 -9.65624632 -5.53180984]; a[-5:] = [ 3.68063076  8.35832876 -7.56794955  1.2929343  -2.51223156]
b[:5] = [ 9.3021517   7.84995943  3.47547217 -5.85633487 -0.94587999]; b[-5:] = [-8.56374418  3.58405984 -1.78723017  9.97203578  5.02214159]
a*b[:5] = [-51.84564555 -16.9606634    8.454743    56.55021204   5.23242822]; a*b[-5:] = [-31.5199803   29.95675047  13.52566776  12.89318715 -12.61678262]


## Multiplicando vetores (GPU)

### Declarando a função

In [25]:
from numba import vectorize

@vectorize(['float64(float64, float64)'], target='cuda')
def mulVectorsGPU(a:np.float64, b:np.float64):
    return a*b

### Rodando e cronometando a função

Agora, avliando o **tempo de execução** da função **`mulVectorsGPU()`**, que roda desta vez na GPU.

Em **dez execuções** de teste (com o mesmo input), temos:

- Tempo médio: ~0,3287 segundos
- Tempo mínimo: ~0,2087 segundos
- Tempo máximo: ~0,5848 segundos

In [26]:
print('Multiplying vectors...')

startT = timer() # Startng our timer
ab = mulVectorsGPU(a, b)
endT = timer() # Endng our timer
execTime = endT - startT

print(f'Vectors multiplied with success!\nExecution time of {execTime} seconds\n\nResults:')

print(f'a[:5] = {a[:5]}; a[-5:] = {a[-5:]}')
print(f'b[:5] = {b[:5]}; b[-5:] = {b[-5:]}')
print(f'a*b[:5] = {ab[:5]}; a*b[-5:] = {ab[-5:]}')

Multiplying vectors...
Vectors multiplied with success!
Execution time of 1.1822460539988242 seconds

Results:
a[:5] = [-5.57351108 -2.16060523  2.43268902 -9.65624632 -5.53180984]; a[-5:] = [ 3.68063076  8.35832876 -7.56794955  1.2929343  -2.51223156]
b[:5] = [ 9.3021517   7.84995943  3.47547217 -5.85633487 -0.94587999]; b[-5:] = [-8.56374418  3.58405984 -1.78723017  9.97203578  5.02214159]
a*b[:5] = [-51.84564555 -16.9606634    8.454743    56.55021204   5.23242822]; a*b[-5:] = [-31.5199803   29.95675047  13.52566776  12.89318715 -12.61678262]


# Analisando Ganhos

Comparando os tempos de execução entre a multiplicação de vetores executada na CPU e a executada na GPU, vemos um ganho imenso de velocidade!

| — | **Tempo de execução (CPU)** | **Tempo de execução (GPU)** | **Speed-up** |
| --- | --- | --- | --- |
| **Média** | 14,6612s | 0,3287s | 44,60x |
| **Mínimo** | 11,6958s | 0,2087s | 56,04x |
| **Máximo** | 22,6586s | 0,5848s | 38,74x |


<!-- | --- | --- | --- |
| --- | --- | --- |
| --- | --- | --- |
| --- | --- | --- | -->